In [1]:
from models.rnn.rnn_plain import RNN_Plain
from models.rnn.rnn_attention import RNN_Attention

from lang import load_data_int_seq
from utils import accuracy_score
%load_ext autoreload
%autoreload 2

C:\Users\danie\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aexploration (use `wandb login --relogin` to force relogin)


True

In [3]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

## Plain RNN

In [4]:
my_config ={"symbols": "+*-0123456789t", 
"output_sequence_length": 9, 
"encoded_seq_length": 9, 
"num_epochs": 100, 
"input_size": input_lang.n_words, 
"hidden_size": 256, 
"output_size": output_lang.n_words, 
"embedding_size": 256, 
"batch_size": 32, 
"num_gru_layers": 2,
"dropout_prob": 0.3,
"calc_magnitude_on":False}

In [5]:
wandb.init(project="test-jupyter-projo",  config=my_config)

In [6]:
algo = RNN_Plain(**my_config)

EncoderRNN(
  (embedding): Embedding(4300, 256)
  (gru): GRU(256, 256, num_layers=2, dropout=0.3)
)
DecoderRNN(
  (embedding): Embedding(16, 256)
  (gru): GRU(256, 256, num_layers=2, dropout=0.3)
  (out): Linear(in_features=256, out_features=16, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [7]:
algo.train(input_lang, output_lang, train)

0m 7s (- 1m 6s) (10 10%) 2.6407
0m 9s (- 0m 39s) (20 20%) 2.4831
0m 12s (- 0m 29s) (30 30%) 2.4604
0m 15s (- 0m 23s) (40 40%) 2.4498
0m 18s (- 0m 18s) (50 50%) 2.4184
0m 20s (- 0m 13s) (60 60%) 2.3876
0m 23s (- 0m 10s) (70 70%) 2.3749
0m 26s (- 0m 6s) (80 80%) 2.3674
0m 28s (- 0m 3s) (90 90%) 2.3332
0m 31s (- 0m 0s) (100 100%) 2.3448


In [ ]:
pred = algo.infer(input_lang, output_lang, X_test[:1000])
pred[:10]

In [ ]:
accuracy_score(pred, y_test[:1000])
if wandb.run is not None:
    wandb.finish()

In [9]:
import test_rnn_plain

### Plain RNN with Magnitude loss

In [ ]:
magn_algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
magn_algo.train(input_lang, output_lang, train)

In [ ]:
pred = magn_algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

##  Attention-based

In [ ]:
algo_attn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=False)
algo_attn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Attention with magnitude loss

In [ ]:
algo_attn_magn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
algo_attn_magn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn_magn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

# Helper Stuff

In [ ]:
import torch
import torch.nn as nn
m = nn.LogSoftmax()
input = torch.randn(2, 3)
print(input)
output = m(input)
print(output)

In [ ]:
# 2D loss example (used, for example, with image inputs)
N, C = 5, 4
loss = nn.NLLLoss()

# input is of size N x C x height x width
data = torch.randn(N, 16, 10, 10)
conv = nn.Conv2d(16, C, (3, 3))
m = nn.LogSoftmax(dim=1)

# print(data.shape)
activated = m(conv(data))
print("output: ", activated.shape)

# each element in target has to have 0 <= value < C
target = torch.empty(N, 8, 8, dtype=torch.long).random_(0, C)
print("target: ", target.shape)
output = loss(activated, target)

# output.backward()

In [ ]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
# input = minibatchsize, number of categories

input = torch.randn(3, 5, requires_grad=True)
# print(input.shape)
activation = m(input)
print(activation.shape)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
print(target.shape)
output = loss(activation, target)
# output.backward()

In [ ]:
from line_profiler import LineProfiler

stuff_to_evaluate = [1,2,3]
def function_to_evaluate(stuff_to_evaluate):
    pass

lp = LineProfiler()
lp_wrapper = lp(function_to_evaluate)
lp_wrapper(stuff_to_evaluate)
lp.print_stats()